In [1]:
# Gerekli kütüphaneleri projemize dahil ediyoruz
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern, graycomatrix, graycoprops
from tqdm import tqdm
import joblib

print("Süper Zengin Öznitelik (HOG + Renk + LBP + GLCM + Augmentation) Çıkarma İşlemi Başlatılıyor...")

# --- 1. Proje Ayarları ve Veri Yolları ---
### DEĞİŞTİRİLDİ: Lütfen bu yolu kendi bilgisayarınızdaki final_dataset1 klasörünün tam yolu ile güncelleyin ###
BASE_PATH = "C:/Users/mkasl/Desktop/donem5/yap470/Ara_rapor_ 1/final_dataset1"

### DEĞİŞTİRİLDİ: Hangi seti işlemek istediğinizi buradan seçin ('train', 'test', veya 'valid') ###
SET_TO_PROCESS = 'test' 

### YENİ: Sadece bu sınıflar işlenecek ###
CLASSES_TO_PROCESS = ['bird', 'drone']

# Kaynak ve çıktı yolları otomatik olarak ayarlanacak
SOURCE_PATH = os.path.join(BASE_PATH, SET_TO_PROCESS)
### DEĞİŞTİRİLDİ: Çıktı dosyasının adı 2 sınıfa göre güncellendi ###
OUTPUT_FILE = f'hog_color_lbp_glcm_augmented_{SET_TO_PROCESS}_kucuk_2class.pkl'


IMAGE_SIZE = (128, 128)
# HOG parametreleri
PIXELS_PER_CELL = (8, 8)
CELLS_PER_BLOCK = (2, 2)
ORIENTATIONS = 9
# Renk Histogramı için 'bin' sayısı
BINS = 8
# LBP PARAMETRELERİ
LBP_POINTS = 24
LBP_RADIUS = 8
# GLCM Ayarları
GLCM_DISTANCES = [1, 2]
GLCM_ANGLES = [0, np.pi/4, np.pi/2, 3*np.pi/4]
GLCM_PROPERTIES = ['energy', 'contrast', 'homogeneity', 'correlation']
GLCM_LEVELS = 32


# --- 2. Yardımcı Fonksiyonlar ---
def augment_color_image(bgr_image):
    augmented_images = []
    augmented_images.append(bgr_image)
    flipped_image = cv2.flip(bgr_image, 1)
    augmented_images.append(flipped_image)
    hsv = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v_brighter = cv2.add(v, 40)
    brighter_hsv = cv2.merge([h, s, v_brighter])
    brighter_image = cv2.cvtColor(brighter_hsv, cv2.COLOR_HSV2BGR)
    augmented_images.append(brighter_image)
    return augmented_images

def extract_color_histogram(image, bins=(BINS, BINS, BINS)):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

def extract_lbp_features(gray_image):
    lbp = local_binary_pattern(gray_image, LBP_POINTS, LBP_RADIUS, method="uniform")
    (lbp_hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, LBP_POINTS + 3), range=(0, LBP_POINTS + 2))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-6)
    return lbp_hist

def extract_glcm_features(gray_image):
    img_for_glcm = (gray_image / 255.0 * (GLCM_LEVELS - 1)).astype(np.uint8)
    glcm = graycomatrix(img_for_glcm, GLCM_DISTANCES, GLCM_ANGLES, levels=GLCM_LEVELS, symmetric=True, normed=True)
    glcm_features = [graycoprops(glcm, prop).flatten() for prop in GLCM_PROPERTIES]
    return np.concatenate(glcm_features)


# --- 3. Öznitelik Çıkarma ve Birleştirme Döngüsü ---
features = []
labels = []

print(f"\n'{SET_TO_PROCESS}' seti işleniyor. Kaynak: {SOURCE_PATH}")

for class_name in tqdm(CLASSES_TO_PROCESS, desc="Sınıflar işleniyor"):
    class_path = os.path.join(SOURCE_PATH, class_name)
    if not os.path.isdir(class_path):
        print(f"Uyarı: '{class_name}' klasörü '{SOURCE_PATH}' içinde bulunamadı, atlanıyor.")
        continue

    for image_name in tqdm(os.listdir(class_path), desc=f"'{class_name}' sınıfı", leave=False):
        image_path = os.path.join(class_path, image_name)
        try:
            original_color_image = cv2.imread(image_path)
            if original_color_image is None: continue
            images_to_process = augment_color_image(original_color_image)
            for color_image in images_to_process:
                color_image_resized = cv2.resize(color_image, IMAGE_SIZE)
                gray_image_resized = cv2.cvtColor(color_image_resized, cv2.COLOR_BGR2GRAY)
                
                hog_features = hog(gray_image_resized, orientations=ORIENTATIONS, pixels_per_cell=PIXELS_PER_CELL, cells_per_block=CELLS_PER_BLOCK, block_norm='L2-Hys')
                color_features = extract_color_histogram(color_image_resized)
                lbp_features = extract_lbp_features(gray_image_resized)
                glcm_features = extract_glcm_features(gray_image_resized)
                
                combined_features = np.concatenate([hog_features, color_features, lbp_features, glcm_features])
                features.append(combined_features)
                labels.append(class_name)
        except Exception as e:
            print(f"Hata: {image_path} işlenemedi. Detay: {e}")

print("\nSüper zengin öznitelik çıkarma tamamlandı!")

# --- 4. Çıkarılan Zengin Özellikleri Kaydetme ---
features = np.array(features, dtype=np.float32)
labels = np.array(labels)
print(f"Toplam {len(features)} adet görüntü için öznitelik vektörü oluşturuldu.")
print(f"Yeni birleşik öznitelik vektörlerinin boyutu: {features.shape[1]}")
data_to_save = {'features': features, 'labels': labels}
joblib.dump(data_to_save, OUTPUT_FILE)
print(f"\nHarika! Tüm birleşik öznitelikler '{OUTPUT_FILE}' dosyasına kaydedildi.")

Süper Zengin Öznitelik (HOG + Renk + LBP + GLCM + Augmentation) Çıkarma İşlemi Başlatılıyor...

'test' seti işleniyor. Kaynak: C:/Users/mkasl/Desktop/donem5/yap470/Ara_rapor_ 1/final_dataset1\test


Sınıflar işleniyor: 100%|██████████| 2/2 [00:52<00:00, 26.02s/it]


Süper zengin öznitelik çıkarma tamamlandı!
Toplam 1698 adet görüntü için öznitelik vektörü oluşturuldu.
Yeni birleşik öznitelik vektörlerinin boyutu: 8670

Harika! Tüm birleşik öznitelikler 'hog_color_lbp_glcm_augmented_test_kucuk_2class.pkl' dosyasına kaydedildi.
